# Deliverable 3. Create a Travel Itinerary Map

----

1. Create a folder called `Vacation_Itinerary` to store all the files for this deliverable.

2. Download the `Vacation_Itinerary_starter_code.ipynb` file into your `Vacation_Itinerary` folder and rename it `Vacation_Itinerary.ipynb`.

3. Make sure the initial dependencies and the Geoapify API key are imp3orted.

4. From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`.

5. Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

    * The point's size should be the maximum temperature for the city

    * The point's color should be the city's name

    * Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

6. From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

    > **Hint:** You will start and end the route in the same city, so the `vacation_start` and `vacation_end` DataFrames will be in the same city.

7. Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details.

8. Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`.

    > **Hint:** You'll use this DataFrame to create a map using GeoViews, so recall that the first column should be the longitude, and the second the latitude.

9. Use GeoViews to create a map that shows the four cities in the itinerary.

10. Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop.

    > **Hint:** You can note that the `mode` parameter is set to `drive`, you can play around with other modes as it's shown in [the "Travel modes" table](https://apidocs.geoapify.com/docs/routing/#api) in the Geoapify Routing API documentation.

11. Use the Geoapify Routing API to retrieve the route's directions for your itinerary.

12. From the JSON response, store the route's legs coordinates in a variable called `legs`.

13. Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude values into two Python lists named `longitude` and `latitude`.

14. Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`.

15. Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map.

16. Use the asterisk operator to display a composed plot that shows the itinerary's route over the map containing the cities. 

17. Save your map to the `Vacation_Itinerary` folder as `WeatherPy_travel_map.png`.

---

In [91]:
# Dependencies and Setup
import geoviews as gv
import geoviews.feature as gf
import hvplot.pandas as hv
import pandas as pd
import requests
from cartopy import crs
from bokeh.models import HoverTool

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv('../Vacation_Search/WeatherPy_vacation.csv')
vacation_df.head()

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Hervey Bay,AU,76.26,broken clouds,-25.2986,152.8535,Hervey Bay Hotel
1,Sampit,ID,72.10,overcast clouds,-2.5333,112.9500,Hotel Sultan
2,Fort Payne,US,68.61,scattered clouds,34.4442,-85.7197,Holiday Inn Express & Suites Fort Payne
3,Gwadar,PK,76.35,light rain,25.1216,62.3254,Sadaf Resorts
4,Lakes Entrance,AU,63.16,light rain,-37.8811,147.9810,Banjo Patterson Motor Inn


## Use GeoViews to create a map that shows all the cities in the `vacation_df` DataFrame. Configure the map as follows:

* The point's size should be the maximum temperature for the city

* The point's color should be the city's name

* Use the `hover_cols` parameter to the the "Hotel Name", "Country", and "Current Description" columns to each point as additional information.

In [3]:
city_map = vacation_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Max Temp",
    scale=1,
    color="City",
    hover_cols=["Hotel Name", "Country", "Weather Description"]
)

# Show the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Hotel Name,Country,Weather Description)

## From the map, *choose four cities* that a customer might want to visit. They should be close together and in the same country. Use the `loc` method to create separate DataFrames for each city on the travel route.

In [4]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method
vacation_start = vacation_df.loc[vacation_df["City"] == "Danville"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Elizabeth City"]
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Wilmington"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Havelock"]
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "North Myrtle Beach"]

## Use the Pandas `concat` function to merge the DataFrame from each city in the itinerary to create a new DataFrame named `itinerary_df` to store the itinerary details

In [20]:
# Use the Pandas concat function to create a new DataFrame to store the itinerary details.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], ignore_index=True)
itinerary_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Danville,US,65.03,overcast clouds,36.5860,-79.3950,Holiday Inn Express Danville
1,Wilmington,US,68.95,few clouds,34.2257,-77.9447,Hell's Kitchen
2,Havelock,US,64.35,overcast clouds,34.8791,-76.9013,Quality Inn Havelock Hwy 70
3,North Myrtle Beach,US,71.38,clear sky,33.8160,-78.6800,Mar Vista Grande
4,Elizabeth City,US,61.05,clear sky,36.2946,-76.2511,Quality Inn


## Use the Pandas `copy` function to create a new DataFrame named `waypoints_df` to store the longitude and latitude for each city in `itinerary_df`

In [32]:
# Create a Pandas DataFrame to store the latitude and longitude for each city in the itineray
waypoints_df = itinerary_df[["Lat", "Lng"]].copy()
waypoints_df

,Lat,Lng
0,36.5860,-79.3950
1,34.2257,-77.9447
2,34.8791,-76.9013
3,33.8160,-78.6800
4,36.2946,-76.2511


## Use GeoViews to create map that shows the four cities in the itinerary

In [98]:
points = gv.Points(itinerary_df, ['Lng', 'Lat'])
hover = HoverTool(tooltips=[('City', '@City'), ('Latitude', '@Lat'), ('Longitude', '@Lng')])

# create a map with the Points element and set the zoom and center
map = gv.tile_sources.EsriNatGeo() * points.opts(color='yellow', size=10, tools=['hover'])
map.opts(width=600, height=400, xaxis=None, yaxis=None, show_grid=False)

# display the map
map

map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp,Weather Description,Hotel Name)

In [45]:
# Set the initial parameters for the API request
api_url = "https://api.geoapify.com/v1/routing"
mode = "drive"

# Create an empty list to store the waypoints
waypoints = []

# Loop through the cities in the itinerary and fetch their coordinates
for index, row in itinerary_df.iterrows():
    city = row["City"]
    country = row["Country"]
    query = f"{city}, {country}"
    params = {"text": query, "apiKey": geoapify_key}
    response = requests.get(f"https://api.geoapify.com/v1/geocode/search?{'&'.join([f'{key}={value}' for key, value in params.items()])}")
    result = response.json()["features"][0]
    longitude, latitude = result["geometry"]["coordinates"]
    waypoints.append((longitude, latitude))

In [46]:
waypoints

[(-87.6304614, 40.125222),
 (-75.546589, 39.7459468),
 (-76.9014457, 34.8785),
 (-78.6800323, 33.8278405),
 (-76.2247697, 36.2956836)]

## Next, you'll use the Geoapify Routing API to find a route between the cities in the itinerary. Review the code that sets the initial parameters and fetches the coordinates from each city to define the `waypoints` parameter by using a `for` loop

In [47]:
# Set parameters to trace the route
radius = 5000
params = {
    "mode":"drive",
    "apiKey": geoapify_key,
}

In [48]:
# Set an empty waypoints String variable
waypoints = ""

# Iterate through the route_df DataFrame to define the waypoints
for index, row in waypoints_df.iterrows():
    waypoints = waypoints + str(row["Lat"]) + "," + str(row["Lng"]) + "|"

# Delete the last character from the string
waypoints = waypoints[:-1]

# Add the waypoints to the params dictionary
params["waypoints"] = waypoints

# Display the params dictionary
params

{'mode': 'drive',
 'apiKey': '7f622a9fd20446c785de0b9af0827018',
 'waypoints': '36.586,-79.395|34.2257,-77.9447|34.8791,-76.9013|33.816,-78.68|36.2946,-76.2511'}

## Use the Geoapify Routing API to retrieve the route's directions for your itinerary

In [50]:
# Set up the base URL for the Geoapify Places API.
base_url = "https://api.geoapify.com/v1/routing"

# Make request and retrieve the JSON data by using the params dictionaty
route_response = params = {
    "waypoints": waypoints,
    "mode": "drive",
    "apiKey": geoapify_key
}

# Convert the API response to JSON format
route_response = requests.get(base_url, params=params).json()

## From the JSON response, store the route's legs coordinates in a variable called `legs`

In [57]:
# Fetch the route's legs coordinates from the JSON reponse
legs = route_response['features'][0]['geometry']['coordinates']
legs

[[[-79.394985, 36.586015],
  [-79.395261, 36.586189],
  [-79.395724, 36.585687],
  [-79.395827, 36.585574],
  [-79.395921, 36.585473],
  [-79.396116, 36.58527],
  [-79.396776, 36.584579],
  [-79.39707, 36.584312],
  [-79.397374, 36.584061],
  [-79.397639, 36.583839],
  [-79.398641, 36.582969],
  [-79.398949, 36.582708],
  [-79.399285, 36.582427],
  [-79.400033, 36.581784],
  [-79.400208, 36.581633],
  [-79.400494, 36.5814],
  [-79.400707, 36.581237],
  [-79.400887, 36.581109],
  [-79.400993, 36.58103],
  [-79.401079, 36.58097],
  [-79.401226, 36.58087],
  [-79.401351, 36.580785],
  [-79.40148, 36.580694],
  [-79.401568, 36.580633],
  [-79.402161, 36.580217],
  [-79.402722, 36.579805],
  [-79.402829, 36.579725],
  [-79.403178, 36.579488],
  [-79.403208, 36.579468],
  [-79.403243, 36.579444],
  [-79.403291, 36.579385],
  [-79.403536, 36.579144],
  [-79.403609, 36.579056],
  [-79.403746, 36.578892],
  [-79.403949, 36.57864],
  [-79.404033, 36.578509],
  [-79.404148, 36.578335],
  [-79.404

## Loop through the route legs coordinates to fetch the latitude and longitude for each step. Store the latitude and longitude value into two Python lists names `longitude` and `latitude`

In [59]:
# Create and empty list to store the longitude of each step
longitude = []

# Create and empty list to store the latitude of step
latitude = []

# Loop through the legs coordinates to fetch the latitude and longitude for each step
for leg in legs:
    for step in leg:
        longitude.append(step[0])
        latitude.append(step[1])

## Use the `longitude` and `latitude` Python lists to create a new DataFrame named `route_df`

In [62]:
for step in legs:
    print(step, type(step))

[[-79.394985, 36.586015], [-79.395261, 36.586189], [-79.395724, 36.585687], [-79.395827, 36.585574], [-79.395921, 36.585473], [-79.396116, 36.58527], [-79.396776, 36.584579], [-79.39707, 36.584312], [-79.397374, 36.584061], [-79.397639, 36.583839], [-79.398641, 36.582969], [-79.398949, 36.582708], [-79.399285, 36.582427], [-79.400033, 36.581784], [-79.400208, 36.581633], [-79.400494, 36.5814], [-79.400707, 36.581237], [-79.400887, 36.581109], [-79.400993, 36.58103], [-79.401079, 36.58097], [-79.401226, 36.58087], [-79.401351, 36.580785], [-79.40148, 36.580694], [-79.401568, 36.580633], [-79.402161, 36.580217], [-79.402722, 36.579805], [-79.402829, 36.579725], [-79.403178, 36.579488], [-79.403208, 36.579468], [-79.403243, 36.579444], [-79.403291, 36.579385], [-79.403536, 36.579144], [-79.403609, 36.579056], [-79.403746, 36.578892], [-79.403949, 36.57864], [-79.404033, 36.578509], [-79.404148, 36.578335], [-79.404321, 36.578074], [-79.404413, 36.577911], [-79.404703, 36.577242], [-79.404

In [63]:
# Create an empty DataFrame to store the steps' coordinates
route_df = route_df = pd.DataFrame(columns=['longitude', 'latitude'])

# Add the steps' longitude and latitude from each step as columns to the DataFrame
for step in legs:
    for lon, lat in step:
        route_df = route_df.append({'longitude': lon, 'latitude': lat}, ignore_index=True)

# Display sample data
route_df.head()

,longitude,latitude
0,-79.394985,36.586015
1,-79.395261,36.586189
2,-79.395724,36.585687
3,-79.395827,36.585574
4,-79.395921,36.585473


##  Use the GeoViews `Path` function to configure a line plot by using `route_df`. Set a custom color and width for the line that may contrast with the map

In [72]:
# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df, kdims=['longitude', 'latitude'])
route_points = gv.Points(route_df, ['longitude', 'latitude'])

In [100]:
import geoviews as gv
import geoviews.tile_sources as gvts

# Set the basemap to be used for the map plot
tiles = gv.tile_sources.EsriNatGeo()

# Configure the route path by using the GeoViews' Path function
route_path = gv.Path(route_df, ['longitude', 'latitude']).opts(color='blue', line_width=2)

# Create the map plot by composing the tiles and route_path objects
route_map = tiles * route_path * map

# Display the map plot
route_map.opts(width=800, height=600)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Path.I   :Path   [longitude,latitude]
   .WMTS.II  :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp,Weather Description,Hotel Name)